# Small CNN — MNIST Classification with Observer

In [3]:
import logging
import os
import sys
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# observer.py lives in the parent directory (neural_network/)
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("__file__"), "..")))
from observer import Observer, ObserverConfig

## Configuration & Hyperparameters

In [4]:
batch_size = 64
num_epochs = 5
lr = 1e-3
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

seed = 42
torch.manual_seed(seed)

print(f"Device: {device}")

Device: cpu


## Observer Setup

In [ ]:
observer_config = ObserverConfig(
    track_profiler=True,
    profile_every_n_steps=100,  # profile every 100th step (0, 100, 200, ...)
    track_memory=True,
    track_throughput=True,
    track_loss=True,
    track_console_logs=True,
    track_error_logs=True,
    track_hyperparameters=True,
    track_system_resources=True,
    track_layer_graph=True,
)

observer = Observer(
    api_key="demo-key",
    project_id="small-cnn-mnist",
    config=observer_config,
)

observer.log_hyperparameters({
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "learning_rate": lr,
    "optimizer": "Adam",
    "dataset": "MNIST",
    "seed": seed,
    "device": device,
})

## Dataset

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

train_dataset = datasets.MNIST("data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST("data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset):,}")
print(f"Test samples:     {len(test_dataset):,}")
print(f"Batches per epoch: {len(train_loader)}")

100.0%
100.0%
100.0%
100.0%


Training samples: 60,000
Test samples:     10,000
Batches per epoch: 938


## Model Definition

In [7]:
class SmallCNN(nn.Module):
    """
    Small convolutional network for MNIST.

    Architecture:
      Conv2d(1,16,3,pad=1) -> ReLU -> MaxPool(2)
      Conv2d(16,32,3,pad=1) -> ReLU -> MaxPool(2)
      Flatten -> Linear(32*7*7, 128) -> ReLU -> Linear(128, 10)

    forward(x, targets=None) returns (logits, loss) to match
    Observer.profile_step() which calls model(x, y).
    """

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x, targets=None):
        x = self.pool(F.relu(self.conv1(x)))  # (B, 16, 14, 14)
        x = self.pool(F.relu(self.conv2(x)))  # (B, 32, 7, 7)
        x = x.view(x.size(0), -1)             # (B, 32*7*7)
        x = F.relu(self.fc1(x))               # (B, 128)
        logits = self.fc2(x)                   # (B, 10)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits, targets)

        return logits, loss

In [8]:
model = SmallCNN().to(device)

num_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {num_params:,}")

observer.register_model(model)

[Observer] Model registered | 206,922 params (0.21M) | 4 param layers


Total parameters: 206,922


## Training

In [9]:
@torch.no_grad()
def evaluate(model, loader):
    """Compute average loss and accuracy on a DataLoader."""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        total_loss += loss.item() * x.size(0)
        correct += (logits.argmax(dim=1) == y).sum().item()
        total += x.size(0)
    model.train()
    return total_loss / total, correct / total

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

print("Starting training...")
training_start = time.time()

for epoch in range(num_epochs):
    for step, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)

        if observer.should_profile(step):
            logits, loss = observer.profile_step(model, x, y)
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
        else:
            logits, loss = model(x, y)
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

        observer.step(epoch, step, loss, batch_size=x.size(0))

    # Validation at end of each epoch
    val_loss, val_acc = evaluate(model, test_loader)
    epoch_report = observer.end_epoch(epoch, val_metrics={
        "val_loss": val_loss,
        "val_acc": val_acc,
    })

    elapsed = time.time() - training_start
    print(
        f"Epoch {epoch}: "
        f"train_loss={epoch_report['loss']['train_mean']:.4f}  "
        f"val_loss={val_loss:.4f}  val_acc={val_acc:.4f}  "
        f"({elapsed:.1f}s)"
    )

training_time = time.time() - training_start
print(f"\nTraining completed in {training_time:.2f}s ({training_time/60:.2f} min)")

Starting training...


c:\Users\admin\.pyenv\pyenv-win\versions\3.10.4\lib\site-packages\torch\profiler\profiler.py:217: UserWarning: Warning: Profiler clears events at the end of each cycle.Only events from the current cycle will be reported.To keep events across cycles, set acc_events=True.
  _warn_once(
f:\hackeurope-2026\neural_network\observer.py:707: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()
[Observer] --- Epoch 0 started ---


## Evaluation

In [ ]:
test_loss, test_acc = evaluate(model, test_loader)
print(f"Final test loss:     {test_loss:.4f}")
print(f"Final test accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

## Observer Report

In [ ]:
report = observer.export(os.path.join("observer_reports", f"{observer.run_id}.json"))

# ── Print summary ──
summary = report["summary"]
print("=" * 60)
print("OBSERVER SUMMARY")
print("=" * 60)
print(f"Total epochs recorded:  {summary.get('total_epochs', 0)}")
print(f"Total training time:    {summary.get('total_duration_s', 0):.2f}s")

if "loss_trend" in summary:
    lt = summary["loss_trend"]
    print(f"\nLoss trend:")
    print(f"  First epoch:  {lt['first']:.4f}")
    print(f"  Last epoch:   {lt['last']:.4f}")
    print(f"  Best:         {lt['best']:.4f}")
    print(f"  Improved:     {lt['improved']}")

if "avg_tokens_per_sec" in summary:
    print(f"\nAvg throughput:  {summary['avg_tokens_per_sec']:.0f} tokens/sec")

if "profiler_highlight" in summary:
    ph = summary["profiler_highlight"]
    print(f"\nProfiler highlight:")
    print(f"  Top operation:       {ph.get('top_op', 'N/A')}")
    print(f"  Top op % of total:   {ph.get('top_op_pct', 0):.1f}%")
    print(f"  Fwd/Bwd time ratio:  {ph.get('fwd_bwd_ratio', 'N/A')}")

# ── Print per-epoch profiler categories ──
print("\n" + "=" * 60)
print("PROFILER: OPERATION CATEGORIES (last epoch)")
print("=" * 60)
for epoch_rec in reversed(report["epochs"]):
    if "profiler" in epoch_rec:
        cats = epoch_rec["profiler"].get("operation_categories", {})
        for cat_name, cat_data in sorted(cats.items(), key=lambda x: -x[1]["cpu_time_ms"]):
            print(f"  {cat_name:<20s}  {cat_data['cpu_time_ms']:>8.1f}ms  ({cat_data['pct_cpu']:>5.1f}%)")
        break

print("=" * 60)
print(f"Full report saved to: observer_reports/{observer.run_id}.json")

observer.close()